In [ ]:
!pip install scikit-learn

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 k

In [ ]:
import pandas as pd
from peft import PeftModel
from transformers import (
    AutoTokenizer,
    GenerationConfig,
    GPTQConfig,
    AutoModelForCausalLM,
)
import torch
from torch import nn
from openai import OpenAI
import openai
import os
import time
from PyPDF2 import PdfReader
import tensorboard
from auto_gptq import exllama_set_max_input_length


#to get into app folder...

%cd drive/MyDrive/app/code


#from app.code.help import load_base_model

/content/drive/MyDrive/app/code


In [ ]:
os.getcwd()

'/content/drive/MyDrive/app/code'

In [ ]:
# %% --------------------------------------------------------------------------

#remember for this script need to havemodified mistral_modelling.py file


def load_base_model(base_model):
    base_path = base_model  # input: base model

    tokenizer = AutoTokenizer.from_pretrained(base_path)
    tokenizer.pad_token = "<unk>"
    tokenizer.padding_side = "right"

    quantization_config_loading = GPTQConfig(
        bits=4,
        #disable_exllama=True, #because led to inrease in inference time by 10 times..
        tokenizer=tokenizer,
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_path,
        return_dict=True,
        quantization_config=quantization_config_loading,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    #model = exllama_set_max_input_length(model, max_input_length=33400)

    tokenizer.pad_token = "<unk>"
    tokenizer.padding_side = "right"
    model.resize_token_embeddings(len(tokenizer))
    model.config.eos_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.pad_token_id

    model.pad_token_id = tokenizer.pad_token_id
    model.eos_token_id = tokenizer.eos_token_id #these 2 lines are diff to usual?

    #model.config.use_return_dict = True #so changes to source code work.

    return tokenizer,model

class app:
    def __init__(self,base_model_path,book):
        self.book = book
        self.base_model_path = base_model_path
        self.model = None
        self.tokenizer = None
        self.fpclass_head = None
        self.contclass_head = None
        self.causal_head = None
        self.page_lag = None
        self.contents_first = None
        self.contents_last = None
        self.contents = None #make pd dataframe but get last contents page first lol.
        self.chapter_breakdown = None


    def get_base(self):
        self.tokenizer, self.model = load_base_model(self.base_model_path)
        self.causal_head = self.model.lm_head #no finetuning was done to causal head, unlike classificaiton head, so can take the base model LM_head for base and causal.

    def load_fpclass_adapter(self):
        self.model.num_labels = 2
        self.model.config.num_labels = 2 #model attributes needed for classificaiton forward pass
        self.model.score = nn.Linear(self.model.config.hidden_size, self.model.config.num_labels, bias=False,dtype=torch.float16) #bias is false as finetuned with and without and made no diff so saves memory
        self.model.load_adapter(peft_model_id=r'../adapters/firstpage',adapter_name='ident_fistpage_adapter')
        self.fpclass_head = self.model.score
        return None

    def load_contclass_adapter(self):
        self.model.num_labels = 2
        self.model.config.num_labels = 2 #model attributes needed for classificaiton forward pass
        self.model.score = nn.Linear(self.model.config.hidden_size, self.model.config.num_labels, bias=False,dtype=torch.float16) #bias is false as finetuned with and without and made no diff so saves memory
        self.model.load_adapter(peft_model_id=r'seatond/identcontents_rank16_lr2.2e-05_target3_39steps_laplha32_batch1_gradacc4',adapter_name='ident_cont_adapter')
        self.contclass_head = self.model.score
        return None

    def load_splitcausal_adapter(self):
        self.model.lm_head = self.causal_head
        self.model.load_adapter(peft_model_id='seatond/revamped_rank64_batch4',adapter_name='splitcausal_adapter') #should make the adapter paths etc dynamic

    def load_extrcontcausal_adapter(self):
        self.model.lm_head = self.causal_head
        self.model.load_adapter(peft_model_id='seatond/revamped_rank64_batch4',adapter_name='extrcontcausal_adapter')

    def change_adapter(self,desired): #desired should be classification,causal,base
        if desired == 'fpclass':
            self.model.lm_head = None #replaced LM_head with score which maps to our 2 classes
            self.model.config.lm_head = None  #remove mpapping to vocab size
            self.model.conditioner = 'classification' #added if statement in source code to direct to correct type of forward pass for classification.
            self.model.score = self.fpclass_head #sets the 2 class head
            self.model.enable_adapters()
            self.model.set_adapter('ident_fistpage_adapter')
        if desired == 'contclass':
            self.model.lm_head = None #replaced LM_head with score which maps to our 2 classes
            self.model.config.lm_head = None  #remove mpapping to vocab size
            self.model.conditioner = 'classification' #added if statement in source code to direct to correct type of forward pass for classification.
            self.model.score = self.contclass_head #sets the 2 class head
            self.model.enable_adapters()
            self.model.set_adapter('ident_cont_adapter')
        if desired == 'splitcausal':
            self.model.score = None
            self.model.config.score = None
            self.model.conditioner = 'causal'
            self.model.lm_head = self.causal_head
            self.model.enable_adapters()
            self.model.set_adapter('splitcausal_adapter')
        if desired == 'extrcontcausal':
            self.model.score = None
            self.model.config.score = None
            self.model.conditioner = 'causal'
            self.model.lm_head = self.causal_head
            self.model.enable_adapters()
            self.model.set_adapter('extrcontcausal_adapter')
        if desired == 'base':
            self.model.score = None
            self.model.config.score = None
            self.model.conditioner = 'causal'
            self.model.lm_head = self.causal_head #looks similar to causal as base is a mistralforcausal LM
            self.model.disable_adapters()


    def get_contents(self):
            self.change_adapter('contclass')
            prompt = """<s>[INST] @@@ Instructions:
        It is your task to classify whether a string corresponds to the contents page of a pdf book.
        A contents page includes chapter titles and page numbers.
        Only reply with the words "Yes" or "No"
        You must reply "yes" if the string is from the contents page, and "no" if it is not the contents page.

        @@@ Question:
        This is the string: ### """
            generation_config = GenerationConfig(
                do_sample=True,
                top_p=0.95, top_k=40,
                temperature=0.7,
                max_new_tokens=150,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.pad_token_id,
            )

            for i in range(0,30):
                input_ids = self.tokenizer(prompt+self.book.pages[i].extract_text()+' ### [/INST]',return_tensors='pt').input_ids.cuda()
                input_ids = input_ids.to("cuda:0") #making sure not on CPU as causes error
                self.model = self.model.to("cuda:0")
                with torch.no_grad(): #no grad as dont want to change any weights just doing an inference
                    predicted_class = torch.argmax(
                        self.model(input_ids).logits
                        ).item() # the inner bit of argmax just gets the logits, we then take max value to get predicted class
                    #no need for softmax as only 2 classes.
                if predicted_class == 1:
                    self.contents_first = i
                    break
            for i in range(self.contents_first,30):
                input_ids = self.tokenizer(prompt+self.book.pages[i].extract_text()+' ### [/INST]',return_tensors='pt').input_ids.cuda()
                input_ids = input_ids.to("cuda:0") #making sure not on CPU as causes error
                self.model = self.model.to("cuda:0")
                with torch.no_grad(): #no grad as dont want to change any weights just doing an inference
                    predicted_class = torch.argmax(
                        self.model(input_ids).logits
                        ).item()
                if predicted_class == 0:
                    self.contents_last = i-1
                    break
            #could make 2 things above a funciton to reduce repetition?
            return 'found contents page .. hopefully'

    def extract_contents(self):
        self.change_adapter('extrcontcausal')
        contents_pages = "" #if method too long can feed separate prompts to LLM for each page of contents page.
        for i in range(self.contents_first,self.contents_last+1):
            contents_pages += self.book.pages[i].extract_text()

        prompt = """<s>[INST] @@@ Instructions:
It is your task to extract the chapters and corresponding page numbers from a string which was created from the contents page of a pdf book.
You must return a list of the chapters and page numbers.
Put each chapter and its page number on its own line, and separate chapters titles from page numbers with a "---".
You will be penalised for separating chapters with anything that is not "---"
For example the first 2 chapters of a contents page should be in the following format: "chapter 1 title --- chapter 1 page number \n chapter 2 title --- chapter 2 page number"

@@@ Question:
string which was created from the contents page of a pdf book: ### """

        input_ids = self.tokenizer(prompt+book.pages[i].extract_text()+' ### [/INST]',return_tensors='pt').input_ids.cuda()
        output = self.tokenizer.decode((self.model.generate(inputs=input_ids, temperature=0.07, do_sample=True, top_p=0.35, top_k=5, max_new_tokens=400000))[0])

        #return output

        contents_list = [i.split('---') for i in output.split('\n')]
        contents_df = pd.DataFrame(contents_list,columns=['chapter_titles','page_number'])

        mask = contents_df['page_number'].notna()
        contents_df = contents_df[mask]
        contents_df['page_number'] = contents_df['page_number'].apply(lambda x: x.replace(' ',''))
        mask = contents_df['page_number'].apply(lambda x: str(x).isnumeric())
        contents_df = contents_df[mask]
        contents_df.reset_index(inplace=True)
        contents_df.drop(columns=['index'],inplace=True)
        contents_df['page_number'] = contents_df['page_number'].apply(lambda x: int(x))
        self.contents = contents_df

        return contents_df

    def find_first_page(self):
        self.change_adapter('fpclass')
        self.model = self.model.to("cuda:0")
        for i in range(self.contents_last+1,self.contents_last+30): #gonna loop through pages until find first page of chapter
            #each iteration will do a forward pass through the model with adapters with page i book
            prompt = f"""<s>[INST] This is a string from a page of a pdf book: ### {self.book.pages[i].extract_text()} ###
            Is it true or false that this page belongs to a chapter called: ### {self.contents['chapter_titles'][0]} ###? [/INST]"""
            input_ids = self.tokenizer(prompt,return_tensors='pt').input_ids.to("cuda:0")
            input_ids = input_ids.to("cuda:0") #making sure not on CPU as causes error

            with torch.no_grad(): #no grad as dont want to change any weights just doing an inference
                predicted_class = torch.argmax(
                    self.model(input_ids).logits
                    ).item() # the inner bit of argmax just gets the logits, we then take max value to get predicted class
                #no need for softmax as only 2 classes

            if predicted_class == 1:
                self.page_lag = i - (self.contents['page_number'][0]) #getting diff betweeen contents page of title and actual page
                break
            else:
                None

        self.contents['corrected_page_number'] = self.contents['page_number'].apply(lambda x: x + self.page_lag)

        #could make 2 things above a funciton to reduce repetition?
        return 'found contents page .. hopefully'

###########################################retreival part done now

    def split_chosen_chapter(self,chosen_chapter): #if results come out bad can do the thing where chop off end of page and put into next one so dont get crap endings.
        self.change_adapter('splitcausal')
        self.model = self.model.to("cuda:0")
        chosen_chapter_index = self.contents[self.contents['chapter_titles'] == chosen_chapter].index[0] #gets row of chosen chapter in contents df
        responses = ""
        for i in range(self.contents.iloc[chosen_chapter_index,2],self.contents.iloc[chosen_chapter_index,2]+2): #between the page numbers of chosen chapter and next chapter
            prompt = f"""[INST]You must split text up into subsections and add informative titles for each subsection.
Each subsection must be in paragraph form and all information should be included from the original text.
You will be penalized for removing information from the original text.
Mark each title you create by adding the symbols "@@@" before each title and placing the title on its own line.
An example subsection format is "@@@title \n content", where you should add the subsection title and content.
This is the text:
### {self.book.pages[i].extract_text()} ### [/INST]
Output: """

            input_ids = self.tokenizer(prompt,return_tensors='pt').input_ids.to("cuda:0")
            input_ids = input_ids.to("cuda:0")

            generation_config = GenerationConfig(
              do_sample=True,
              top_p=0.95, top_k=40,
              temperature=0.7,
              max_new_tokens=150000,
              eos_token_id=self.tokenizer.eos_token_id,
              pad_token_id=self.tokenizer.pad_token_id,
            )


            out = self.model.generate(inputs=input_ids, generation_config=generation_config)
            decoded_output = self.tokenizer.decode(out[0], skip_special_tokens=True)
            decoded_output = decoded_output[
                decoded_output.find("[/INST]")
                + len("[/INST]"):
            ]

            responses += decoded_output


        responses = responses.split('@@@')[1:]
        self.chapter_breakdown = pd.DataFrame({
            'subtitle':list(map(lambda x: x[:x.find('\n')],responses)),
            'text':list(map(lambda x: x[x.find('\n')+2:],responses))})

##currently base_model and peft model are mixed up, choose a way which doesnt do memory in.
#havent set pad tpoken, try with and without will i need to re finetune??
# warning about pad token and also attnetnion mask, do we need it?








In [ ]:
#test classification score..

def prep_data(test_data_path):
    data = pd.read_json(test_data_path)
    df = pd.DataFrame(
        {
            "content": data.apply(
                lambda x: f"""<s>[INST] This is a string from a page of a pdf book: ### {x['page']} ###
                Is it true or false that this page belongs to a chapter called: ### {x['chapter_title']} ###? [/INST]""",
                axis=1,
            ),
            'label': data['label']
        }
    )
    #return data_aq
    return df

def inference(tokenizer,model,test_data):
    ground = []
    pred = []
    model = model.to("cuda:0")
    for i in range(0,38):
        print(i)
        X_test_tokenized = tokenizer(test_data['content'][i], return_tensors="pt").input_ids.to("cuda:0")
        with torch.no_grad():
            logits = model(X_test_tokenized).logits

        # Convert logits to probabilities using softmax
        probs = torch.nn.functional.softmax(logits, dim=-1)

        # Get the predicted class index
        predicted_class = torch.argmax(probs).item()
        predicted_class = 'yes' if predicted_class == 1 else 'No'

        pred.append(predicted_class)
        ground.append(test_data['label'][i])
        print(i)
    return pd.DataFrame({'ground':ground,'pred':pred})

from auto_gptq import exllama_set_max_input_length
obj.model = exllama_set_max_input_length(obj.model, max_input_length=23010)

obj.set_adapter('classification')
test_data_path = r"../../Colab Notebooks/firstpage_testdata.json"
test_data = prep_data(test_data_path)
results = inference(obj.tokenizer,obj.model,test_data)
results

In [ ]:
#logits test
def prep_data(test_data_path):
    data = pd.read_json(test_data_path)
    df = pd.DataFrame(
        {
            "content": data.apply(
                lambda x: f"""<s>[INST] This is a string from a page of a pdf book: ### {x['page']} ###
                Is it true or false that this page belongs to a chapter called: ### {x['chapter_title']} ###? [/INST]""",
                axis=1,
            ),
            'label': data['label']
        }
    )
    #return data_aq
    return df
def logits_test(obj,test_data,tokenizer,model):
    model = model.to("cuda:0")
    input_ids = tokenizer(test_data['content'][20], return_tensors="pt").input_ids.to("cuda:0")
    with torch.no_grad(): #no grad as dont want to change any weights just doing an inference
        log = obj.model(input_ids).logits
    return log


from auto_gptq import exllama_set_max_input_length
obj.model = exllama_set_max_input_length(obj.model, max_input_length=23010)
obj.set_adapter('classification')
test_data_path = r"../../Colab Notebooks/firstpage_testdata.json"
test_data = prep_data(test_data_path)
logits_test(obj,test_data,obj.tokenizer,obj.model)


tensor([[10.1016, -0.9805]], device='cuda:0', dtype=torch.float16)

In [ ]:
#test fp ident
def prep_data(test_data_path):
    df = pd.read_csv(test_data_path)
    return df
def inference(tokenizer,model3,test_data):
    ground = []
    pred = []
    for i in range(0,len(test_data)):
        print(i)
        X_test_tokenized = tokenizer(test_data['x_test'][i], return_tensors="pt").input_ids.to("cuda:0")
        X_test_tokenized = X_test_tokenized.to("cuda:0") #making sure not on CPU as causes error
        model = model3.to("cuda:0")
        with torch.no_grad(): #no grad as dont want to change any weights just doing an inference
            predicted_class = torch.argmax(
                model(X_test_tokenized).logits
                ).item()

        pred.append(predicted_class)
        ground.append(test_data['y_test'][i])
        print(i)

    # %%
    return pd.DataFrame({'ground':ground,'pred':pred})


test_data_path = r'../../contents_ident_and_extract/data/classification_test_data.csv'
test_data = prep_data(test_data_path)
results = inference(obj.tokenizer,obj.model,test_data)

In [ ]:
base_model_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
#book = PdfReader(r"../../edexcel_a_level_physics_student_book_1.pdf")
book = PdfReader(r"../../business book for testing.pdf")

obj = app(base_model_path,book)
obj.get_base()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pret

In [ ]:
mod,tok = obj.model,obj.tokenizer

In [ ]:
obj.load_fpclass_adapter()
obj.load_contclass_adapter()
obj.load_splitcausal_adapter()
obj.load_extrcontcausal_adapter()

In [ ]:
get_contents(obj)

'found contents page .. hopefully'

In [ ]:
obj.contents_last

4

In [ ]:
#obj.model,obj.tokenizer = mod,tok
obj.contents_first,obj.contents_last = 2,4

In [ ]:
torch.cuda.empty_cache()
hi = 0


In [ ]:
#obj.set_adapter('base')
#obj.contents_first = 2
obj.model.conditioner = 'causal'
get_contents(obj)
#print(obj.contents_first,obj.contents_last)

 yes</s>


'found contents page .. hopefully'

In [ ]:
def extract_contents(self):
        self.set_adapter('base')
        #contents_pages = "" #if method too long can feed separate prompts to LLM for each page of contents page.
        #for i in range(3,4):
            #contents_pages += self.book.pages[i].extract_text()
            #contents_pages = pdf_text
        prompt = """<s>[INST] @@@ Instructions:
It is your task to extract the chapters and corresponding page numbers from a string which was created from the contents page of a pdf book.
You must return a list of the chapters and page numbers.
Put each chapter and its page number on its own line, and separate chapters titles from page numbers with a "---".
You will be penalised for separating chapters with anything that is not "---"
For example the first 2 chapters of a contents page should be in the following format: "chapter 1 title --- chapter 1 page number \n chapter 2 title --- chapter 2 page number"

@@@ Question:
string which was created from the contents page of a pdf book: ### """

        #input_ids = self.tokenizer(prompt+book.pages[i].extract_text()+' ### [/INST]',return_tensors='pt').input_ids.cuda()
        input_ids = self.tokenizer(prompt+pdf_text+' ### [/INST]',return_tensors='pt').input_ids.cuda()
        output = self.tokenizer.decode((self.model.generate(inputs=input_ids, temperature=0.07, do_sample=True, top_p=0.35, top_k=5, max_new_tokens=400000))[0])

        return output

        contents_list = [i.split('---') for i in output.split('\n')]
        contents_df = pd.DataFrame(contents_list,columns=['chapter_titles','page_number'])

        mask = contents_df['page_number'].notna()
        contents_df = contents_df[mask]
        contents_df['page_number'] = contents_df['page_number'].apply(lambda x: x.replace(' ',''))
        mask = contents_df['page_number'].apply(lambda x: str(x).isnumeric())
        contents_df = contents_df[mask]
        contents_df.reset_index(inplace=True)
        contents_df.drop(columns=['index'],inplace=True)
        contents_df['page_number'] = contents_df['page_number'].apply(lambda x: int(x))
        self.contents = contents_df

        return contents_df


In [ ]:
os.getcwd()

'/content/drive/MyDrive/app/code'

In [ ]:
import fitz  # PyMuPDF

def read_pdf(pdf_path):
    doc = fitz.open(pdf_path)

    text = ''

    for page_num in range(3,4):
        page = doc[page_num]
        text += page.get_text()

    doc.close()

    return text

# Example usage
pdf_path = r"../../business book for testing.pdf"
pdf_text = read_pdf(pdf_path)
print(pdf_text)


iv
COURSE STRUCTURE
MEETING 
CUSTOMER NEEDS 2
ENTREPRENEURS 
AND LEADERS 
140
MARKETING MIX 
AND STRATEGY 
56
1.  THE MARKET  
3
2. MARKET RESEARCH  
11
 3.  MARKET POSITIONING  
20
19.  ROLE OF AN  
ENTREPRENEUR 
141
20.  ENTREPRENEURIAL 
MOTIVES AND 
CHARACTERISTICS 
149
21. BUSINESS OBJECTIVES 155
22.  BUSINESS CHOICES 
160
 9.  MARKETING OBJECTIVES  
AND STRATEGY 
57
10.  PRODUCT/SERVICE  
DESIGN 
68
11.  PROMOTION AND  
BRANDING  
74
12. PRICING STRATEGIES 
83
13.  DISTRIBUTION 
 
89
THE MARKET 
30
MANAGING  
PEOPLE  
95
4.  DEMAND 
 
31
5.  SUPPLY 
36
6.  MARKETS 
41
7.  PRICE ELASTICITY OF  
DEMAND (PED) 
46
8.  INCOME ELASTICITY OF  
DEMAND (YED) 
52
14.  APPROACHES TO  
STAFFING 
96
15.  RECRUITMENT,  
SELECTION AND  
TRAINING  
104
16.  ORGANISATIONAL  
DESIGN 
113
17.  MOTIVATION IN THEORY 
AND PRACTICE 
 
 121
18. LEADERSHIP 
 
132
UNIT 1: MARKETING AND PEOPLE
SAMPLE



In [ ]:
print(obj.book.pages[3].extract_text())

iv COURSE STRUCTURE
MEETING 
CUSTOMER NEEDS  2ENTREPRENEURS AND LEADERS
 140MARKETING MIX AND STRATEG 
Y 56
1.  THE MARKET  3
2. MARKET RESEARCH  11
 3.  MARKET POSITIONING  2019.  ROLE OF AN  
ENTREPRENEUR  141
20.  ENTREPRENEURIAL MOTIVES AND CHARA
CTERISTICS  149
21. BUSINESS OBJECTIVES  155
22.  BUSINESS CHOICES  160 9.  MARKETING OBJECTIVES  AND STRATEG 
Y 57
10.  PRODUCT/SERVICE  
DESIGN  68
11.  PROMOTION AND  BRANDING
  74
12. PRICING STRATEGIES  83
13.  DISTRIBUTION   89
THE MARKET  30 MANAGING  
PEOPLE   95
4.  DEMAND   31
5.  SUPPL Y  36
6.  MARKETS  41
7.  PRICE ELASTICITY OF  
DEMAND (PED)  46
8.  INCOME ELASTICITY OF  DEMAND (YED)
 5214.  APPROACHES TO  S 
TAFFING  96
15.  RECRUITMENT,  
SELECTION AND  TRAINING
  104
16.  ORGANISATIONAL  
DESIGN  113
17.  MOTIVATION IN THEOR
Y 
AND PRACTICE    121
18. LEADERSHIP   132UNIT 1: MARKETING AND PEOPLE
M00_Business_FM_39170.indd   4 5/9/18   7:38 PM
SAMPLE


In [ ]:
torch.cuda.empty_cache()
hi = extract_contents(obj)
#obj.extract_contents()


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(hi)

<s><s> [INST] @@@ Instructions:
It is your task to extract the chapters and corresponding page numbers from a string which was created from the contents page of a pdf book.
You must return a list of the chapters and page numbers.
Put each chapter and its page number on its own line, and separate chapters titles from page numbers with a "---".
You will be penalised for separating chapters with anything that is not "---"
For example the first 2 chapters of a contents page should be in the following format: "chapter 1 title --- chapter 1 page number 
 chapter 2 title --- chapter 2 page number"

@@@ Question:
string which was created from the contents page of a pdf book: ### iv
COURSE STRUCTURE
MEETING 
CUSTOMER NEEDS 2
ENTREPRENEURS 
AND LEADERS 
140
MARKETING MIX 
AND STRATEGY 
56
1.  THE MARKET  
3
2. MARKET RESEARCH  
11
 3.  MARKET POSITIONING  
20
19.  ROLE OF AN  
ENTREPRENEUR 
141
20.  ENTREPRENEURIAL 
MOTIVES AND 
CHARACTERISTICS 
149
21. BUSINESS OBJECTIVES 155
22.  BUSINESS CHOI

In [ ]:
print("1. Marketing and People --- 5/9/18 7:38 PM --- 4\n2. The Market --- 14\n3. Market Research --- 11\n---\n4. Market Positioning --- 2019.9\n5. Role of an Entrepreneur --- 141\n---\n6. Entrepreneurial Motives and Characteristics --- 149\n7. Business Objectives --- 155\n---\n8. Business Choices --- 160\n9. Marketing Objectives and Strategy --- 57.9\n---\n10. Product/Service Design --- 68\n11. Promotion and Branding --- 74\n---\n12. Pricing Strategies --- 83\n13. Distribution --- 89\n---\n14. Managing People --- 95\n15. Demand --- 31\n---\n16. Supply --- 36\n---\n17. Markets --- 41\n---\n18. Price Elasticity of Demand (PED) --- 46\n---\n19. Income Elasticity of Demand (YED) --- 52\n---\n20. Approaches to Staffing --- 96\n---\n21. Recruitment, Selection and Training --- 104\n---\n22. Organizational Design --- 113\n---\n23. Motivation in Theory and Practice --- 121\n---\n24. Leadership --- 13")

1. Marketing and People --- 5/9/18 7:38 PM --- 4
2. The Market --- 14
3. Market Research --- 11
---
4. Market Positioning --- 2019.9
5. Role of an Entrepreneur --- 141
---
6. Entrepreneurial Motives and Characteristics --- 149
7. Business Objectives --- 155
---
8. Business Choices --- 160
9. Marketing Objectives and Strategy --- 57.9
---
10. Product/Service Design --- 68
11. Promotion and Branding --- 74
---
12. Pricing Strategies --- 83
13. Distribution --- 89
---
14. Managing People --- 95
15. Demand --- 31
---
16. Supply --- 36
---
17. Markets --- 41
---
18. Price Elasticity of Demand (PED) --- 46
---
19. Income Elasticity of Demand (YED) --- 52
---
20. Approaches to Staffing --- 96
---
21. Recruitment, Selection and Training --- 104
---
22. Organizational Design --- 113
---
23. Motivation in Theory and Practice --- 121
---
24. Leadership --- 13


In [ ]:
obj.find_first_page()

'found contents page .. hopefully'

In [ ]:
obj.page_lag

9

In [ ]:
def split_chosen_chapter(self,chosen_chapter_index): #if results come out bad can do the thing where chop off end of page and put into next one so dont get crap endings.
        self.set_adapter('causal')
        self.model = self.model.to("cuda:0")
        responses = ""
        #for i in range(self.contents.iloc[chosen_chapter_index,2],self.contents.iloc[chosen_chapter_index,2]+2): #between the page numbers of chosen chapter and next chapter
        for i in range(53,54):
            prompt = f"""<s>[INST]You must split text up into subsections and add informative titles for each subsection.
Each subsection must be in paragraph form and all information should be included from the original text.
You will be penalized for removing information from the original text.
Mark each title you create by adding the symbols "@@@" before each title and placing the title on its own line.
An example subsection format is "@@@title \n content", where you should add the subsection title and content.
This is the text:
### {self.book.pages[i].extract_text()} ### [/INST]
Output: """

            input_ids = self.tokenizer(prompt,return_tensors='pt').input_ids.to("cuda:0")
            input_ids = input_ids.to("cuda:0")

            generation_config = GenerationConfig(
              do_sample=True,
              top_p=0.95, top_k=40,
              temperature=0.7,
              max_new_tokens=1500,
              eos_token_id=self.tokenizer.eos_token_id,
              pad_token_id=self.tokenizer.pad_token_id,
            )


            out = self.model.generate(inputs=input_ids, generation_config=generation_config)
            decoded_output = self.tokenizer.decode(out[0], skip_special_tokens=True)
            decoded_output = decoded_output[
                decoded_output.find("[/INST]")
                + len("[/INST]"):
            ]

            responses += decoded_output
        return responses

In [ ]:
chapter_num = 0
test = split_chosen_chapter(obj,chapter_num) #so assuming here user has been presented with chapters and then selected the chapter num and then convert this to index by minus 1
test


"\nOutput: \n\n@@@Prior Knowledge for Understanding Momentum (44) \n\nTo fully comprehend the concept of momentum as presented in this chapter, you should:\n\n/uni279C have a solid grasp of the principles of rectilinear motion (Chapter 3).\n/uni279C understand Newton's second and third laws of motion.\n/uni279C be familiar with the difference between scalar and vector quantities.\n/uni279C comprehend the meaning of mass.\n/uni279C know Newton's equation of motion: F = ma.\n\nBefore proceeding, test your knowledge with the following tasks:\n\n1. Arrange the following physical quantities into scalar or vector quantities: acceleration, displacement, distance, mass, speed, time, and velocity.\n2. Explain the difference between mass and weight.\n3. Calculate the resultant force acting on a 2.0 kg mass falling through a liquid if the resistive force is 12 N.\n4. What is the resultant force when forces of 3.0 N and 4.0 N act at right angles at a point?\n5. Calculate the resultant force acting

In [ ]:
test

In [ ]:
retrieve = retriever(book,model,tokenizer)
#retrieve.get_contents()

In [ ]:
retrieve.contents_first=4
retrieve.contents_last =4
print(retrieve.extract_contents())

 chapter 1: introduction --- iv
chapter 2: get the most from this book --- vi
chapter 3: introduction ---
chapter 1: quantities and units --- 1
chapter 2: practical skills --- 10

chapter 3: mechanics
chapter 3.1: rectilinear motion --- 25
chapter 3.2: momentum --- 44
chapter 3.3: forces --- 55
chapter 3.4: work, energy and power --- 73

chapter 4: electric circuits
chapter 4.1: charge and current --- 90
chapter 4.2: potential difference, electromotive force and power --- 102
chapter 4.3: current–potential difference relationships --- 116
chapter 4.4: resistance and resistivity --- 128
chapter 4.5: internal resistance, series and parallel circuits, and the potential divider --- 145

chapter 5: materials
chapter 5.1: fluids --- 173
chapter 5.2: solid materials --- 188

chapter 6: waves and the particle behaviour of light
chapter 6.1: nature of waves --- 210
chapter 6.2: transmission and reflection of waves --- 221
chapter 6.3: superposition of waves --- 247
chapter 6.4: particle nature 

In [ ]:
retrieve.contents_first

4

In [ ]:
retrieve.contents_last

4

In [ ]:
from auto_gptq import exllama_set_max_input_length
model = exllama_set_max_input_length(model, max_input_length=6000)

In [ ]:
title = 'Quantities and units'
string = '1 1.1 Physical quantities, base and derived units \n1.1 Physical quantities, base \nand derived units\nAn elderly physicist was asked how much he had in the bank.\n‘How much of what?’ he responded.\n‘Money, of course!’\n‘Fifteen million, three hundred thousand, one hundred and four,’ he replied.\nThe physicist was not a rich man. He had quoted his balance in Turkish \nLira which, at that time, had an exchange rate of 2.6 million to the pound \n(1.4 million to the American dollar).\nThe story has relevance to measurements in physics. It is meaningless to state that \nthe size of a wire is 10; we must state the quantity that is measured (in this case,\xa0the \nlength of the wire) and the unit (such as cm). In this chapter, and throughout this \nbook, you will identify and use a number of base quantities  (and their units) \nthat are fundamental to all physical measurements. You will develop and use \nderived units for quantities for a wide range of physical properties.Prior knowledge\nIn this chapter you will need to:\n/uni279C use common measures and simple compound measures such as speed \n/uni279C substitute values into formulae and equations using appropriate units and \nrearrange equations in order to change the subject. \nThe key facts that will be useful are:\n/uni279C mass, length and time are examples of measurable physical quantities\n/uni279C kilogram (kg), metre (m) and second (s) are units of mass, length and time\n/uni279C speed is the distance covered per unit time and is measured in metres per \nsecond (m s–1)\n/uni279C vector quantities have both size and direction. 1 Quantities and units\nTest yourself on prior knowledge\n1 A man walks 1.6 km in 20 minutes. Calculate his average speed.\n2 A sprinter runs 200 m at an average speed of 8.0 m s –1. Calculate the \ntime taken for her to complete the distance.\n3 Acceleration can be calculated by dividing a change in speed by the \ntime taken. State the unit of acceleration.\n4 Speed is a scalar quantity and velocity is a vector. Explain the \ndifference between the two.\nTip\nMany students lose marks in \nexaminations by failing to include the \nunit of a derived quantity! Always show \nthe unit for all calculated quantities.\n807527_C01_Edexcel_Physics_001-009.indd   1 24/02/2015   14:06www.ebook3000.com\n'
prompt = f"""<s>[INST] @@@ Instructions:
You are an assisstant who must classify whether a string from a page of a pdf book corresponds to the first page of a given chapter in that book.
You will be given the string and also given the chapter title.
The first page of a chapter usually contains the name of the chapter towards the start of the string.
The first word of you answer must be "Yes" or "No"
You must reply "yes" if the string is the first page of the given chapter, and "no" if it is not the first page of the given chapter.

@@@ Example:
User: The given chapter title is ### The Chemistry Of Life ### and the string is: ### 'CHEMISTRY FOR BIOLOGISTS 61A.1 THE CHEMISTRY OF LIFE\nTHE CHEMISTRY OF WATER\nAll reactions in living cells take place in water. Without water, \nsubstances could not move around the body. Water is one of  the reactants in the process of  photosynthesis, on which almost all life depends (see fig E). Understanding the properties of  water will help you understand many key systems in living organisms. \nWater is also a major habitat – it supports more life than any other \npart of  the planet.\n▲ fig E  W ater is vital for life on Earth in many different ways – in a desert, \nthe smallest amount of water allows plants to grow.\nThe simple chemical formula of  water is H2O. This tells us that \ntwo atoms of  hydrogen are joined to one atom of  oxygen to make \nup each water molecule. However, because the electrons are held closer to the oxygen atom than to the hydrogen atoms, water is a polar molecule (see fig F).\n104.5°Oδ2\nHδ1Hδ1\n▲ fig F  A model of a w ater molecule showing dipoles.\nOne major effect of  this polarity is that water molecules form hydrogen  bonds. The slightly negative oxygen atom of  one water \nmolecule will attract the slightly positive hydrogen atoms of  other water molecules in a weak electrostatic attraction called a hydrogen \nbond. Each individual hydrogen bond is weak but there are many of  them so the molecules of  water ‘stick together’ more than you might expect (see fig G). Water has relatively high melting and boiling points compared with other substances that have molecules of  a similar size because it takes a lot of  energy to break all the hydrogen bonds that hold the molecules together. Hydrogen bonds are important in protein structure (see Sections 1A.5 and 2B.1) \nand in the structure and functioning of  DNA (see Section 2B.3).Oδ2\nOδ2\nHδ1Hδ1Hδ1Hδ1\nHδ1Oδ2Oδ2\nOδ2Hδ1\nHδ1\nHδ1Hδ1Hδ1\n▲ fig G  Hydr ogen bonding in water molecules, based on attraction \nbetween positive and negative dipoles.\nTHE IMPORTANCE OF WATER\nThe properties of  water make it very important in biological \nsystems for many reasons.\n •W ater is a polar solvent. Because it is a polar molecule, many ionic \nsubstances like sodium chloride will dissolve in it (see fig H).  \nMany covalently bonded substances are also polar and will dissolve in water, but often do not dissolve in other covalently bonded solvents such as ethanol. Water also carries other substances, such as starch. As a result, most of  the chemical reactions within cells occur in water (in aqueous solution).\nsodium and chloride ionsin solution in water\nsalt and water mixed\nsodium chloride\nNaClionic bond sodium ionchlorideionδ1 chargeson hydrogenin water areattracted tonegativechloride ion\nδ2\n charges\non oxygenin water ar\ne\nattracted tothe positivesodium ionH\nH\nHHH\nH HO\nOO\nCl2\nCl2\nCl2Cl2\nCl2\nNa1OH\nCl2\nNa1Na1\nCl2Cl2\nCl2\nCl2Cl2\nNa1Na1Na1 Na1\nNa1Na1Na1\n▲ fig H  A model of sodium chloride dissolving in water as a result of the \ninteractions between the charges on sodium and chloride ions and the dipoles of the water molecules.\nUncorrected proof, all content subject to change at publisher discretion. Not for resale, circulation or distribution in whole or in part. ©Pearson 2018' ###
Assisstant: "Yes"

@@@ Example:
User: The given chapter title is ### Preparing For Your Exams ### and the string is: ### 'xASSESSMENT OVERVIEW\nPAPER / UNIT 1PERCENTAGE \nOF IASPERCENTAGE OF IALMARK TIME AVAILABILITY\nMOLECULES, DIET, TRANSPORT AND \nHEALTH \nWritten examination\nPaper code \nWBI11/01\nExternally set and marked by \nPearson Edexcel\nSingle tier of entry40% 20% 80 1 hour  \n30 minutesJanuary, June and October\nFirst assessment : January 2019\nPAPER / UNIT 2PERCENTAGE \nOF IASPERCENTAGE OF IALMARK TIME AVAILABILITY\nCELLS, DEVELOPMENT, BIODIVERSITY \nAND CONSERVATION\nWritten examination\nPaper code \nWBI12/01\nExternally set and marked by \nPearson Edexcel\nSingle tier of entry40% 20% 80 1 hour  \n30 minutesJanuary, June and October\nFirst assessment : June 2019\nPAPER / UNIT 3PERCENTAGE \nOF IASPERCENTAGE OF IALMARK TIME AVAILABILITY\nPRACTICAL SKILLS IN BIOLOGY 1   \nWritten examination\nPaper code \nWBI13/01\nExternally set and marked by \nPearson Edexcel\nSingle tier of entry20% 10% 50 1 hour  \n20 minutesJanuary, June and October\nFirst assessment : June 2019ASSESSMENT OVERVIEW\nThe following tables give an overview of the assessment for Pearson Edexcel International Advanced Subsidiary course \nin Biology. You should study this information closely to help ensure that you are fully prepared for this course and know exactly what to expect in each part of the examination. More information about this qualification, and about the question types in the different papers, can be found on page 302 of this book.\nUncorrected proof, all content subject to change at publisher discretion. Not for resale, circulation or distribution in whole or in part. ©Pearson 2018' ###
Assisstant: "No"

@@@ Question:
User: The given chapter title: ### {title} ### This is the string: ### {string} ###
[/INST]"""


input_ids = tokenizer(prompt,return_tensors='pt').input_ids.cuda()
output = tokenizer.decode((model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=5))[0])
output[output.find('[/INST]')+len('[/INST]'):].lower()

' yes</s>'